In [1]:
import json
import pandas as pd
from sql_queries import ValorantQueries

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [4]:
vq.aggregate_gamestate(5)

,Timestamp,EventType,ATK_Loadout_Value,ATK_Shield,ATK_Weapon_Price,ATK_Initiator,ATK_Duelist,ATK_Controller,ATK_Sentinel,DEF_Loadout_Value,DEF_Shield,DEF_Weapon_Price,DEF_Initiator,DEF_Duelist,DEF_Controller,DEF_Sentinel
0,0,start,24000,250,14500,1,2,1,1,23250,200,16600,2,1,1,1
1,72891,kill,24000,250,14500,1,2,1,1,18550,150,13700,2,1,1,0
2,73216,kill,18500,200,11600,1,2,1,0,18550,150,13700,2,1,1,0
3,80084,kill,13900,150,8700,0,2,1,0,18550,150,13700,2,1,1,0
4,80351,kill,13900,150,8700,0,2,1,0,12900,125,8700,2,0,1,0
5,86246,kill,13900,150,8700,0,2,1,0,8300,75,5800,1,0,1,0
6,88979,kill,13900,150,8700,0,2,1,0,3700,25,2900,0,0,1,0
7,92160,plant,13900,150,8700,0,2,1,0,3700,25,2900,0,0,1,0
8,95954,kill,13900,150,8700,0,2,1,0,0,0,0,0,0,0,0


### AAA